In [1]:
from astroquery.gaia import Gaia
from astropy import units as u
from astropy.constants import h, c
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display

In [2]:
# Constants

d_filters = {'Band':['G','BP','RP'],'Wavelength':[639,518,783],'Bandpass':[290,169,189]} 
filters = pd.DataFrame(data=d_filters)

ccd_width_px = 4656
ccd_height_px = 3520
ccd_px_side_length_micron = 3.8
ccd_width_micron = ccd_width_px * ccd_px_side_length_micron
ccd_height_micron = ccd_height_px * ccd_px_side_length_micron
plate_scale_arcsec_micron = 0.11
ccd_width_range_arcsec = ccd_width_micron * plate_scale_arcsec_micron
ccd_height_range_arcsec = ccd_height_micron * plate_scale_arcsec_micron
ccd_width_range_hourangle = u.arcsec.to(u.hourangle, value=ccd_width_range_arcsec)
ccd_height_range_hourangle = u.arcsec.to(u.hourangle, value=ccd_height_range_arcsec)
mast_collecting_area_m2 = 0.09 * np.pi * 0.5
gaia_collecting_area_m2 = 0.7

In [3]:
ra_widget = widgets.BoundedFloatText(
    value=81.28,
    min=0,
    max=360,
    step=0.001,
    description='RA:'
)
dec_widget = widgets.BoundedFloatText(
    value=-69.78,
    min=-90,
    max=90,
    step=0.001,
    description='DEC:'
)
max_mag_widget = widgets.FloatText(
    value=18,
    step=0.01,
    description='Max Mag:'
)
display(ra_widget)
display(dec_widget)
display(max_mag_widget)

BoundedFloatText(value=81.28, description='RA:', max=360.0, step=0.001)

BoundedFloatText(value=-69.78, description='DEC:', max=90.0, min=-90.0, step=0.001)

FloatText(value=18.0, description='Max Mag:', step=0.01)

In [4]:
def get_entries(max_mag, ra, dec, ccd_width_range_hourangle, ccd_height_range_hourangle):
    query = '''SELECT TOP 100 RA, DEC,
PHOT_G_MEAN_FLUX AS PHOT_G_MEAN_FLUX_GAIA, PHOT_G_MEAN_FLUX_ERROR AS PHOT_G_MEAN_FLUX_ERROR_GAIA,
PHOT_BP_MEAN_FLUX AS PHOT_BP_MEAN_FLUX_GAIA, PHOT_BP_MEAN_FLUX_ERROR AS PHOT_BP_MEAN_FLUX_ERROR_GAIA,
PHOT_RP_MEAN_FLUX AS PHOT_RP_MEAN_FLUX_GAIA, PHOT_RP_MEAN_FLUX_ERROR AS PHOT_RP_MEAN_FLUX_ERROR_GAIA
FROM gaiadr3.gaia_source
WHERE (ra BETWEEN {ra_min} AND {ra_max})
AND (dec BETWEEN {dec_min} AND {dec_max})
AND phot_g_mean_mag < {max_mag}
'''.format(
        max_mag=max_mag,
        ra_min=ra-(ccd_width_range_hourangle/2), ra_max=ra+(ccd_width_range_hourangle/2),
        dec_min=dec-(ccd_height_range_hourangle/2), dec_max=dec+(ccd_height_range_hourangle/2)
    )
    job = Gaia.launch_job(query)
    r = job.get_results()
    return r

In [5]:
def convert_to_flux_density(table):
    # gets flux mean, flux error in electrons/sec (as given in GAIA_SOURCE) and name of band {'G','BP,'RP'}
    # returns flux density + uncertainty in watt/m^2/nm
    for filter in d_filters['Band']:
        lam0 = filters[filters.Band == filter].iloc[0]['Wavelength']
        bandpass = filters[filters.Band == filter].iloc[0]['Bandpass']
        P = table['phot_' + str.lower(filter) + '_mean_flux_gaia'] * h.value * c.value /lam0 # convert rate of elecron arrival, to incoming power in watts 
        dP = np.sqrt((table['phot_' + str.lower(filter) + '_mean_flux_error_gaia'] * h.value * c.value /lam0)**2  + (bandpass/2 * table['phot_' + str.lower(filter) + '_mean_flux_gaia'] * h.value * c.value /lam0**2)**2)
        F = P/gaia_collecting_area_m2 # convert incoming power, to incoming intensity- 0.7m^2 is GAIA's collecting area
        dF = dP/gaia_collecting_area_m2
        table['flux_' + filter + '_density_gaia'] = (F/bandpass) * (u.watt/u.m**2/u.nm) # convert to incoming flux density
        table['flux_' + filter + '_density_error_gaia'] = dF/bandpass * (u.watt/u.m**2/u.nm)

In [6]:
def calculate_mast_qe(flux_density_inc_mast):
    return flux_density_inc_mast

In [7]:
def convert_gaia_to_mast(table):
    for filter in d_filters['Band']:
        gaia_mast_attenuation = 0.5
        flux_density_inc_mast = table['flux_' + filter + '_density_gaia'] * gaia_mast_attenuation
        flux_density_meas_mast = calculate_mast_qe(flux_density_inc_mast)
        flux_density_meas_mast = flux_density_meas_mast * mast_collecting_area_m2
        table['phot_' + filter + '_flux_mast'] = (flux_density_meas_mast * filters[filters.Band == filter].iloc[0]['Wavelength'] / (h.value * c.value)) * (u.electron / u.s) * (u.m**2 * u.nm / u.watt)

        flux_density_inc_error_mast = table['flux_' + filter + '_density_error_gaia'] * gaia_mast_attenuation
        flux_density_meas_error_mast = calculate_mast_qe(flux_density_inc_error_mast)
        flux_density_meas_error_mast = flux_density_meas_error_mast * mast_collecting_area_m2
        table['phot_' + filter + '_flux_rel_error_mast'] = (((flux_density_meas_error_mast * filters[filters.Band == filter].iloc[0]['Wavelength'] / (h.value * c.value)) * (u.electron / u.s) * (u.m**2 * u.nm / u.watt)) / table['phot_' + filter + '_flux_mast'])*100

In [8]:
entries = get_entries(max_mag_widget.value, ra_widget.value, dec_widget.value, ccd_width_range_hourangle, ccd_height_range_hourangle)
convert_to_flux_density(entries)
convert_gaia_to_mast(entries)
entries

ra,dec,phot_g_mean_flux_gaia,phot_g_mean_flux_error_gaia,phot_bp_mean_flux_gaia,phot_bp_mean_flux_error_gaia,phot_rp_mean_flux_gaia,phot_rp_mean_flux_error_gaia,flux_G_density_gaia,flux_G_density_error_gaia,flux_BP_density_gaia,flux_BP_density_error_gaia,flux_RP_density_gaia,flux_RP_density_error_gaia,phot_G_flux_mast,phot_G_flux_rel_error_mast,phot_BP_flux_mast,phot_BP_flux_rel_error_mast,phot_RP_flux_mast,phot_RP_flux_rel_error_mast
deg,deg,electron / s,electron / s,electron / s,electron / s,electron / s,electron / s,W / (m2 nm),W / (m2 nm),W / (m2 nm),W / (m2 nm),W / (m2 nm),W / (m2 nm),electron / s,,electron / s,,electron / s,
float64,float64,float64,float32,float64,float32,float64,float32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
81.26273980915636,-69.79092056997047,2487.82781274411,5.307263,2525.186905016465,39.877506,2587.294867209299,85.95206,3.8097822582432665e-27,8.645045812915082e-28,8.185698727462544e-27,1.3353118580513225e-27,4.961363565902604e-27,5.987852579537625e-28,0.8662767760987604,22.691705790297345,1.5088330022753143,16.312741312741313,1.3823514536556658,12.068965517241377
81.26289960381898,-69.79126367731901,2093.2417722791606,2.2463803,1016.7978500977687,60.698627,2137.1632909373207,31.15145,3.2055254489041683e-27,7.273884038984421e-28,3.296073194066277e-27,5.376798936266417e-28,4.098197008939213e-27,4.946099838374913e-28,0.7288795168203868,22.691705790297345,0.6075503361047636,16.31274131274131,1.141853145294298,12.068965517241386
81.26308807720777,-69.77267239353029,1619.7717454448598,2.2229776,1232.1149682336957,19.472305,752.2529265523971,13.014775,2.4804681524132002e-27,5.628605353676276e-28,3.994049671143987e-27,6.515389907561138e-28,1.4425105964695416e-27,1.7409610647046193e-28,0.5640143736925738,22.69170579029734,0.7362051985044167,16.312741312741313,0.4019170523296654,12.06896551724138
81.26375568969132,-69.776956229495,2118.95504561264,2.152953,899.4557631435065,18.018543,1885.2278884014609,32.88253,3.24490195721613e-27,7.363236053150843e-28,2.9156946288396206e-27,4.756297222721004e-28,3.615088901338539e-27,4.3630383292016855e-28,0.7378330349908038,22.691705790297334,0.5374368672756925,16.312741312741316,1.0072479735620117,12.068965517241379
81.26562895218676,-69.77449239933273,6311.393107631972,6.563054,4938.966634096495,27.764053,2362.945349811924,20.776604,9.665071418159404e-27,2.1931695706308505e-27,1.601025762148091e-26,2.6117119092956308e-27,4.5311537990337215e-27,5.468633895385526e-28,2.19766546782863,22.691705790297338,2.951098724556719,16.31274131274131,1.2624849918032832,12.068965517241379
81.26581331982938,-69.77335265860012,2474.6191457981536,4.2101507,2264.1929564739244,34.802822,1200.5533586753309,16.678333,3.789554915849243e-27,8.599146522662602e-28,7.339655280850353e-27,1.197298979212075e-27,2.302165774818872e-27,2.7784759351262248e-28,0.8616774379291767,22.69170579029734,1.3528856218367311,16.31274131274131,0.6414370088192128,12.068965517241379
81.26708546365232,-69.7778242439881,3266.847494552198,2.7805705,2440.6550421298584,19.588444,1243.5862638859123,16.450905,5.0027488081674506e-27,1.1352090409769644e-27,7.911678471343554e-27,1.2906116425261202e-27,2.384685123793195e-27,2.8780682528538556e-28,1.1375361675314124,22.691705790297338,1.4583240818410497,16.31274131274131,0.6644288215525782,12.068965517241379
81.26718636625183,-69.79102645338159,2092.763420734292,1.9538122,977.3368796529422,11.160848,1534.4081088227026,12.08011,3.2047929162025452e-27,7.272221797329718e-28,3.16815568629172e-27,5.168130414896725e-28,2.9423613762855425e-27,3.5511257989653105e-28,0.7287129519029888,22.691705790297338,0.5839718776585059,16.312741312741313,0.8198104153547637,12.06896551724138
